## Introdução

O presente notebook aborda o problema de **detecção de fraudes em transações financeiras**. Cada linha da base representa uma transação feita com cartão de crédito, contendo informações como data, valor, localização, estabelecimento, e um campo binário (`is_fraud`) que indica se a transação foi fraudulenta ou não.

Para a tarefa de classificação, foi utilizado uma abordagem híbrida dos algoritmos **Random Forest** e **Isolation Forest**.


## Bibliotecas Utilizadas

| Biblioteca / Módulo                        | Descrição                                                                 |
|-------------------------------------------|---------------------------------------------------------------------------|
| `pandas` (`pd`)                           | Utilizada para manipulação e análise de dados em forma de tabelas (DataFrames). |
| `sklearn.model_selection.train_test_split` | Função para dividir o conjunto de dados em treino e teste.                |
| `sklearn.preprocessing.OneHotEncoder`     | Codifica variáveis categóricas em vetores binários (one-hot encoding).    |
| `sklearn.compose.ColumnTransformer`       | Permite aplicar diferentes transformações a diferentes colunas do DataFrame. |
| `sklearn.pipeline.Pipeline`               | Cria um fluxo de processamento com etapas sequenciais (ex: pré-processamento + modelo). |
| `sklearn.ensemble.RandomForestClassifier` | Algoritmo de aprendizado de máquina baseado em múltiplas árvores de decisão para classificação. |
| `sklearn.ensemble.IsolationForest`        | Algoritmo para detecção de anomalias baseado em árvores (unsupervised).   |
| `numpy` (`np`)                            | Biblioteca para computação numérica e manipulação de arrays.              |
| `sklearn.metrics.*`                       | Conjunto de funções para avaliação de modelos (ex: accuracy, F1, ROC AUC, etc.). |
| `matplotlib.pyplot` (`plt`)               | Biblioteca para criação de gráficos e visualizações.                      |


In [ ]:
# Manipulação de dados
import numpy as np
import pandas as pd

# Pré-processamento e transformação
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
# Modelos
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.pipeline import Pipeline
# Divisão dos dados
from sklearn.model_selection import train_test_split
# Métricas de avaliação
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score
)
# Visualização opcional (se quiser ver distribuição dos scores)
import matplotlib.pyplot as plt

## Descrição das Colunas da Base de Dados

| Coluna                 | Descrição                                                                 |
|------------------------|---------------------------------------------------------------------------|
| `Unnamed: 0`           | Índice original do dataset                                                |
| `trans_date_trans_time` | Data e hora da transação.                                                |
| `cc_num`               | Número do cartão de crédito (anonimizado).                                |
| `merchant`             | Nome do comerciante onde a transação ocorreu.                             |
| `category`             | Categoria do comerciante (ex: food, gas_transport, etc).                  |
| `amt`                  | Valor da transação em dólares.                                            |
| `first`                | Primeiro nome do titular do cartão.                                       |
| `last`                 | Sobrenome do titular do cartão.                                           |
| `gender`               | Gênero do titular do cartão (`M` ou `F`).                                 |
| `street`               | Endereço residencial do titular do cartão.                                |
| `lat`                  | Latitude da residência do titular do cartão.                              |
| `long`                 | Longitude da residência do titular do cartão.                             |
| `city_pop`             | População estimada da cidade de residência.                               |
| `job`                  | Profissão do titular do cartão.                                           |
| `dob`                  | Data de nascimento do titular do cartão.                                  |
| `trans_num`            | Identificador único da transação.                                         |
| `unix_time`            | Timestamp UNIX da transação.                                              |
| `merch_lat`            | Latitude do comerciante.                                                  |
| `merch_long`           | Longitude do comerciante.                                                 |
| `is_fraud`             | Indicador de fraude (`1` para fraude, `0` para legítima).                 |


In [ ]:
# Importação e visualização das colunas
dataframe = pd.read_csv('/kaggle/input/fraud-detection/fraudTrain.csv')
dataframe.head()

## Organização de dados relevantes

1. Transformar datas em  conjuntos de objetos em formato datetime.
2. Organização de transações por cartões, em seguida, calcula a diferença de tempo entre cada transação.
3. Remove colunas irrelevantes, definir coluna de label `is_fraud`, e remover a coluna label dos dados que serão usados no treinamento.
4. Categorizando colunas em classes `Numéricas` e `Categóricas`.

### Separação de Variáveis: `X` e `y`

| Variável | Conteúdo                              | Função no Modelo                                |
|----------|----------------------------------------|--------------------------------------------------|
| `X`      | Todas as colunas **independentes** do DataFrame (features), exceto a variável alvo (`target_col`). | Usada como entrada para o treinamento e predição. |
| `y`      | Coluna **alvo** (`target_col`) do DataFrame, contendo os rótulos (ex: `is_fraud`). | Usada como saída esperada no treinamento (o que o modelo tenta prever). |


In [ ]:
# Datas para Objetos Datetimes
dataframe['trans_date_trans_time'] = pd.to_datetime(dataframe['trans_date_trans_time'])
dataframe['hour_of_day'] = dataframe['trans_date_trans_time'].dt.hour
dataframe['day_of_week'] = dataframe['trans_date_trans_time'].dt.dayofweek
dataframe['month'] = dataframe['trans_date_trans_time'].dt.month

In [ ]:
# Diferença de tempo entre transações.
dataframe = dataframe.sort_values(by=['cc_num', 'trans_date_trans_time'])
dataframe['time_since_last_transaction'] = dataframe.groupby('cc_num')['trans_date_trans_time'].diff().dt.total_seconds().fillna(0)

In [ ]:
# Definir coluna label
target_col = 'is_fraud'
# Removendo colunas irrelevantes e coluna label dos dados de treinamento
columns_to_drop_final = ['Unnamed: 0', 'trans_num', 'first', 'last', 'street', 'city', 'zip', 'dob', 'ssn']
df_processed = dataframe.drop(columns=columns_to_drop_final, errors='ignore').copy()
X = df_processed.drop(columns=[target_col], errors='ignore')
y = df_processed[target_col]
# Categorização de colunas
numerical_cols = ['amt', 'lat', 'long', 'city_pop', 'unix_time', 'time_since_last_transaction', 'merch_lat', 'merch_long']
categorical_cols = ['category', 'gender', 'state', 'job', 'merchant']

## Treinamento

1. Pré-Processamento de dados para o modelo usar de forma eficiente as classes `numéricas` e `categóricas`
2. Separação de treino e testes. Neste caso, foi utilizado 80% dos dados para testes e 20% para validações do modelo.
3. Filtragem de dados com Isolation Forest.
5. Treinamento do modelo utilizando o modelo Random Forest com os dados pré-filtrados pelo IF.

In [ ]:
# Pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='drop'
)

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Pré-filtragem com Isolation Forest
iso = IsolationForest(n_estimators=100, contamination=0.01, random_state=42, n_jobs=-1)
X_train_proc = preprocessor.fit_transform(X_train)
iso.fit(X_train_proc)

# Aplicar no conjunto de teste
X_test_proc = preprocessor.transform(X_test)
anomaly_scores = -iso.decision_function(X_test_proc)

# Selecionar apenas os mais anômalos
threshold = np.percentile(anomaly_scores, 90)
mask_suspects = anomaly_scores >= threshold

# Filtrar para Random Forest
X_test_suspects = X_test[mask_suspects]
y_test_suspects = y_test[mask_suspects]

# Treinar Random Forest no conjunto completo (poderia ser refinado também)
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced'))
])

rf_pipeline.fit(X_train, y_train)

# Aplicar RF apenas nos casos suspeitos

rf_predictions = rf_pipeline.predict(X_test_suspects)
rf_probabilities = rf_pipeline.predict_proba(X_test_suspects)[:, 1]
rf_predictions = (rf_probabilities >= 0.3).astype(int) 

## Avaliação do Modelo

In [ ]:
# Avaliação
print("\nAvaliação apenas nos casos suspeitos (filtrados pelo Isolation Forest):")
print(classification_report(y_test_suspects, rf_predictions, target_names=['Não Fraude', 'Fraude']))

accuracy = accuracy_score(y_test_suspects, rf_predictions)
precision = precision_score(y_test_suspects, rf_predictions, pos_label=1)
recall = recall_score(y_test_suspects, rf_predictions, pos_label=1)
f1 = f1_score(y_test_suspects, rf_predictions, pos_label=1)
roc_auc = roc_auc_score(y_test_suspects, rf_probabilities)
pr_auc = average_precision_score(y_test_suspects, rf_probabilities)

print(f"Acurácia: {accuracy:.4f}")
print(f"Precisão (Fraude): {precision:.4f}")
print(f"Recall (Fraude): {recall:.4f}")
print(f"F1-Score (Fraude): {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")
print(f"AUC-PR: {pr_auc:.4f}")

### Conclusão do Modelo Híbrido: Isolation Forest + Random Forest

#### Análise dos Resultados:

A abordagem híbrida combina um filtro inicial por anomalias (Isolation Forest) com um classificador supervisionado (Random Forest) para refinar os casos suspeitos. Nos dados analisados:

1. A acurácia foi de 99,37%, e o F1-Score para fraude alcançou 82,99% **dentro do subconjunto de transações suspeitas**.

2. O modelo demonstrou bom desempenho interno: das 238 fraudes detectadas, 84,03% foram corretamente identificadas, com 81,97% de precisão.

3. **No entanto, apenas 238 das 1.501 fraudes reais foram analisadas**, o que representa **apenas 15,85% do total de fraudes**. Isso significa que cerca de **84% das fraudes reais foram perdidas já na etapa de filtragem pelo Isolation Forest**.